### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
# from langchain.agents import Tool
from langchain_core.tools import Tool

from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:
# Ensure grpc / grpcio-tools are compatible with generated protos
# This checks the installed grpc version and upgrades to >=1.70.0 if needed.
import sys, subprocess, importlib
from packaging import version

def ensure_grpc():
    try:
        import grpc
        current = version.parse(grpc.__version__)
        required = version.parse("1.70.0")
        if current < required:
            print(f"Upgrading grpcio from {grpc.__version__} to >=1.70.0")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "grpcio>=1.70.0", "grpcio-tools>=1.70.0"])
            importlib.reload(grpc)
    except ModuleNotFoundError:
        print("Installing grpcio>=1.70.0 and grpcio-tools>=1.70.0")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "grpcio>=1.70.0", "grpcio-tools>=1.70.0"])
    except subprocess.CalledProcessError as e:
        raise RuntimeError(
            "Failed to install/upgrade grpc packages. If you are on an older Python (e.g., 3.8/3.9), please upgrade to Python 3.10+ and retry."
        ) from e
    finally:
        import grpc
        print("grpc version:", grpc.__version__)

ensure_grpc()

grpc version: 1.76.0


In [3]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [4]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [5]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [6]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [7]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [8]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [9]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [10]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some compelling reasons to consider using AutoGen in your AI Agent project:

1. **Multi-Agent Communication**: AutoGen facilitates seamless communication between multiple AI agents, allowing them to collaborate effectively. This communication enhances coordination and problem-solving capabilities.

2. **Reduced Coordination Complexity**: By using natural language handoffs, AutoGen simplifies inter-agent protocols, enabling easier coordination among agents without extensive custom development.

3. **Enhanced Functionality Integration**: AutoGen can be easily integrated with various APIs, tools, and external systems. This capability enhances the overall functionality of your AI agents, making them more versatile.

4. **Efficiency and Scalability**: The framework is designed to streamline the orchestration, optimization, and automation of workflows involving large language models (LLMs). This leads to improved efficiency and the ability to scale your applications effectively.

5. **Ease of Use**: AutoGen is created with usability in mind, which can reduce the development time and effort required to implement AI solutions, making it accessible for teams with varying skill levels.

6. **Open-Source Advantage**: Being open-source allows for customization and community engagement, leading to continual improvement and support for the framework.

These advantages make AutoGen a strong candidate for enhancing the capabilities and efficiency of your AI projects. 

TERMINATE

## Cons of AutoGen:
Here are some cons associated with using AutoGen for your AI Agent project:

1. **Limitations in Creativity**: AutoGen generates content based on predefined templates and inputs, which may restrict its ability to produce truly original or creative outputs.

2. **Import Function Issues**: There may be formatting problems when importing files, like PDFs, which can affect the overall user experience and efficiency.

3. **Magic Feature Limitations**: The system might not offer advanced features that some projects might require, which can hinder flexibility in development.

4. **Over-engineering Risks**: The framework can lead to overly complex systems due to its autonomous capabilities, potentially increasing maintenance difficulties and operational risks.

5. **Lack of Visual Builder**: Unlike some competitors, AutoGen does not provide a visual builder or no-code options, which could be a drawback for teams seeking easier integration or customization.

6. **Production Readiness**: AutoGen may not yet be fully ready for real-world production use, raising concerns about reliability and performance in critical applications.

These factors should be considered when deciding whether to implement AutoGen in your project. 

TERMINATE



## Decision:

Based on the pros and cons presented, I recommend using AutoGen for the project. The advantages, particularly in facilitating multi-agent communication, reducing coordination complexity, enhancing functionality integration, and providing scalability make it a strong choice for improving our AI capabilities. Although there are some limitations regarding creativity and potential over-engineering, the overall efficiency and ease of use, alongside the open-source nature, provide a compelling case for its adoption. The concerns about production readiness can be addressed through thorough testing and validation during the implementation phase.

TERMINATE

In [11]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [12]:
await host.stop()